In [2]:
import torch
import torch.nn.utils.rnn as rnn_utils

In [14]:
a = torch.Tensor([1, 2, 3])
b = torch.Tensor([4, 5, 0])
c = torch.Tensor([6, 0, 0])
d = torch.stack([a, b, c],1)
print d
pack = rnn_utils.pack_padded_sequence(d, torch.Tensor([3,2,1]).long(), batch_first=True)
print pack

unpack, lengths = rnn_utils.pad_packed_sequence(pack)

print unpack
print unpack[0]
# out = rnn_utils.pack_padded_sequence([a, b, c], batch_first=True)
# print out

tensor([[1., 4., 6.],
        [2., 5., 0.],
        [3., 0., 0.]])
PackedSequence(data=tensor([1., 2., 3., 4., 5., 6.]), batch_sizes=tensor([3, 2, 1]))
tensor([[1., 2., 3.],
        [4., 5., 0.],
        [6., 0., 0.]])
tensor([1., 2., 3.])


In [59]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
import numpy as np

max_seq_len = 3
batch_size = 4
layer_num = 2

input_size = 10 # 0 - 9
emb_size = 8
hidden_size = 10

# test data:
a = [[1,2,3], [4,5,0], [6,0,0], [7,8,9]]
lens = [3,2,1,3]

# sort the input batch data by reversed actual length for pad_pack operation
pairs = sorted( zip(a, lens), key=lambda p: p[1], reverse=True)
(a, lens) = zip(*pairs)

# actual length
lens = np.array(lens)
# lens = torch.LongTensor(lens)

va = Variable(torch.LongTensor(a))
vlens = Variable(torch.LongTensor(lens))
print 'va, vlen'
print(va, vlens)
print

embedding = nn.Embedding(input_size, emb_size)
gru = nn.LSTM(emb_size, hidden_size, layer_num, batch_first=True)

# print 'va.size'
# print va.size()
# print 

# print 'vlens'
# print vlens
# print 

inputs = va
# inputs = va.transpose(0, 1); 
# print("inputs size: ", inputs.size()) # max_seq_len * batch_size

inputs = embedding(inputs); 
# print("embedded size: ", inputs.size()) # max_seq_len * batch_size * emb_size

packed = torch.nn.utils.rnn.pack_padded_sequence(inputs, lens, batch_first=True)

# print 'packed', 
# print packed
# print 

# h0 = Variable(torch.randn(layer_num, 4, 10))
outputs, (hn, cn) = gru(packed)

print 'lstm------'
print outputs
print
tmp = torch.nn.utils.rnn.pad_packed_sequence(outputs, batch_first=True)

outputs, output_lengths = tmp

print"after padded:"
print outputs.size()
print outputs
print

print "hn"
print hn[-1]
# print("outputs.size: ", outputs.size())
# print("hn size: ", hn.size())



va, vlen
(tensor([[1, 2, 3],
        [7, 8, 9],
        [4, 5, 0],
        [6, 0, 0]]), tensor([3, 3, 2, 1]))

lstm------
PackedSequence(data=tensor([[-0.0677,  0.1141, -0.0320, -0.0092,  0.0023, -0.0765, -0.0856,  0.0674,
         -0.0150,  0.0464],
        [-0.0734,  0.1129, -0.0267, -0.0052, -0.0089, -0.0771, -0.0820,  0.0527,
         -0.0162,  0.0301],
        [-0.0602,  0.1219, -0.0399,  0.0327, -0.0093, -0.0755, -0.0957,  0.0645,
         -0.0271,  0.0274],
        [-0.0694,  0.1156, -0.0351,  0.0109, -0.0053, -0.0845, -0.0892,  0.0660,
         -0.0266,  0.0343],
        [-0.0704,  0.1836, -0.0327, -0.0002,  0.0007, -0.1083, -0.1414,  0.1291,
         -0.0254,  0.0696],
        [-0.0936,  0.1799, -0.0522, -0.0078, -0.0086, -0.1134, -0.1273,  0.1132,
         -0.0249,  0.0555],
        [-0.0835,  0.1808, -0.0400,  0.0294, -0.0118, -0.1048, -0.1439,  0.1123,
         -0.0270,  0.0431],
        [-0.0719,  0.2156, -0.0238,  0.0410, -0.0077, -0.1229, -0.1706,  0.1560,
         -0.03

In [56]:
masks = (vlens-1)
print masks
masks = masks.unsqueeze(0)
print masks
masks = masks.unsqueeze(2)
print masks
masks= masks.expand(max_seq_len, outputs.size(1), outputs.size(2))
# print masks
# output = outputs.gather(0, masks)[0]

tensor([2, 2, 1, 0])
tensor([[2, 2, 1, 0]])
tensor([[[2],
         [2],
         [1],
         [0]]])


RuntimeError: The expanded size of the tensor (3) must match the existing size (4) at non-singleton dimension 1

In [60]:
nn.Variable

AttributeError: 'module' object has no attribute 'Variable'